# Connect to openrouter.ai using openai API and use free LLM models

## Imports

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

from IPython.display import Markdown, display, update_display

## app setup

In [2]:
load_dotenv()

openrouter_base_url = os.getenv("OPENROUTER_BASE_URL")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

openrouter_llm_model = os.getenv("MINIMAX_FREE")

if (openrouter_base_url and openrouter_api_key and openrouter_llm_model) is None:
    print('Openrouter Base url and api key are missing')
else:
    print(f"""Found base url and api key.
          Base URL: {openrouter_base_url}
          API Key: {openrouter_api_key[:10]}
          LLM Model: {openrouter_llm_model}""")
    


Found base url and api key.
          Base URL: https://openrouter.ai/api/v1
          API Key: sk-or-v1-b
          LLM Model: minimax/minimax-m2:free


## Connect to LLM model


In [3]:
client = OpenAI(base_url=openrouter_base_url, api_key=openrouter_api_key)

payload = [
    {'role': 'system', 'content': 'You are a humorous assisant.'},
    {'role': 'user', 'content': 'Tell me a fun fact about lions.'}
]

In [4]:

response = client.chat.completions.create(model=openrouter_llm_model, messages=payload)

print(response.choices[0].message.content)

A male lion’s roar can carry up to 5 miles (8 km), and members of a pride will sometimes roar together—like a wildlife choir that rehearses for audience-free fans 🦁.


Write another prompt and stream the output

In [5]:
payload = [
    {'role': 'system', 'content': 'You are a expert software engineer, having vast experience in DSA ALGO, AI ML, using python, java and Springboot tech stack.'},
    {'role': 'user', 'content': 'Write an algorithm with example for greedy algorithm and explain it for begineers.'}
]

stream_resp = client.chat.completions.create(model=openrouter_llm_model, messages=payload, stream=True)

display_writer = display(Markdown(""), display_id=True)
resp = ""
for chunk in stream_resp:
    resp += chunk.choices[0].delta.content
    update_display(Markdown(resp), display_id=display_writer.display_id)


Here’s a simple, beginner-friendly explanation of greedy algorithms, followed by three classic examples with runnable Python code.

Greedy algorithm at a glance
- Idea: At each step, pick the option that looks best right now (greedy choice), without reconsidering it later.
- When it works: Problems where making the locally best choice also leads to a globally optimal solution. These usually have:
  - Optimal substructure: An optimal solution contains optimal solutions to subproblems.
  - Greedy choice property: The locally optimal choice can be made without affecting future choices.
- When it fails: Problems where a locally optimal choice blocks a better global outcome. Greedy may be fast but incorrect in those cases.

General greedy skeleton
- Sort candidates by some measure.
- Iteratively select candidates that remain feasible.
- Update the remaining problem and continue.

Example 1: Fractional Knapsack (greedy works)
- Sort items by value per unit weight.
- Keep adding items; if the next item doesn’t fit fully, take the fraction you need.

```python
# Fractional Knapsack
# Greedy choice: pick highest value/weight ratio first; take fraction if needed.

from dataclasses import dataclass
from typing import List

@dataclass
class Item:
    name: str
    weight: float
    value: float

def fractional_knapsack(capacity: float, items: List[Item]):
    # Sort by value/weight descending
    items_sorted = sorted(items, key=lambda it: it.value / it.weight, reverse=True)

    total_value = 0.0
    taken = []  # tuples of (name, fraction_taken)

    for item in items_sorted:
        if capacity <= 0:
            break
        if item.weight <= capacity:
            # Take whole item
            total_value += item.value
            capacity -= item.weight
            taken.append((item.name, 1.0))
        else:
            # Take fraction
            fraction = capacity / item.weight
            total_value += item.value * fraction
            taken.append((item.name, round(fraction, 4)))
            capacity = 0
    return total_value, taken

if __name__ == "__main__":
    items = [
        Item("A", 10, 60),
        Item("B", 20, 100),
        Item("C", 30, 120),
    ]
    capacity = 50
    value, taken = fractional_knapsack(capacity, items)
    print("Max value:", value)
    print("Taken:", taken)
```

Example 2: Activity Selection (greedy works)
- Pick the activity that finishes earliest among those you can attend next.

```python
# Activity Selection
# Greedy choice: pick the activity that finishes first among those that start after the last chosen finishes.

from dataclasses import dataclass

@dataclass
class Activity:
    name: str
    start: int
    finish: int

def activity_selection(activities: List[Activity]):
    # Sort by earliest finish time
    activities_sorted = sorted(activities, key=lambda a: a.finish)
    selected = []
    last_finish = -float("inf")

    for a in activities_sorted:
        if a.start >= last_finish:
            selected.append(a)
            last_finish = a.finish
    return selected

if __name__ == "__main__":
    activities = [
        Activity("A", 1, 4),
        Activity("B", 3, 5),
        Activity("C", 0, 6),
        Activity("D", 5, 7),
        Activity("E", 8, 9),
        Activity("F", 5, 9),
    ]
    chosen = activity_selection(activities)
    print("Selected activities:", [a.name for a in chosen])
```

Example 3: Coin Change (greedy does NOT always work)
- Greedy approach: use the largest coin first.
- Counterexample: Denominations [1, 3, 4] to make 6. Greedy picks 4 + 1 + 1 (3 coins), but optimal is 3 + 3 (2 coins).

```python
# Coin Change (greedy does NOT always give optimal results)
# Greedy choice: always use the largest coin possible first.

def coin_change_greedy(amount, coins):
    coins = sorted(coins, reverse=True)
    used = []
    total = 0
    for c in coins:
        cnt = amount // c
        if cnt:
            used.append((c, cnt))
            amount -= c * cnt
            total += cnt
    if amount != 0:
        return None, total, used  # not possible
    return None, total, used

if __name__ == "__main__":
    coins = [1, 3, 4]
    amount = 6
    _, total_coins, used = coin_change_greedy(amount, coins)
    print(f"Greedy uses {total_coins} coins: {used}")  # 3 coins (4+1+1)
    print("But optimal is 2 coins: (3, 3)")
```

Tips for beginners
- Always check if the problem has the greedy choice property; otherwise, consider dynamic programming or other methods.
- Greedy algorithms are usually fast: often O(n log n) due to sorting.
- When in doubt, test on edge cases and known counterexamples.